In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from scipy.stats import randint, uniform
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================
# 0. Cấu hình chung
# ==========================
RANDOM_STATE = 42
DATA_PATH = r"D:\elliptic\blte\Labeled-Transactions-based-Dataset-of-Ethereum-Network-master\FinalDataset.xlsx"
# (hoặc .xlsx nếu bạn dùng bản excel -> dùng read_excel)

In [2]:
# ==========================
# 1. Load final_dataset
# ==========================
df = pd.read_excel(DATA_PATH)

print("Số dòng ban đầu:", len(df))
print("Các cột:", df.columns.tolist())

Số dòng ban đầu: 71250
Các cột: ['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category']


In [ ]:
# ==========================
# 2. Tạo nhãn transaction-level
# ==========================
# Điền thiếu cho from_scam/to_scam (nếu có NaN)
for col in ["from_scam", "to_scam"]:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# 1 = abnormal nếu from hoặc to là scam
df["label"] = (
    (df.get("from_scam", 0) == 1) |
    (df.get("to_scam", 0) == 1)
).astype(int)

LABEL_COL = "label"

print("Phân bố nhãn (0=normal, 1=abnormal):")
print(df[LABEL_COL].value_counts())

y = df[LABEL_COL].values

Phân bố nhãn (0=normal, 1=abnormal):
label
0    57000
1    14250
Name: count, dtype: int64


In [4]:
# ==========================
# 3. Chọn feature (tránh rò rỉ label)
# ==========================

# Các cột KHÔNG dùng làm feature:
drop_id_cols = [
    'block_number',
    'block_timestamp',
    "hash",          # tx hash
    'transaction_index',
    "from_address",  # địa chỉ
    "to_address",
    "block_hash",
    "input"          # raw input data, text dài
]

drop_label_cols = [
    "from_scam", "to_scam",
    "from_category", "to_category",
    LABEL_COL
]

cols_to_drop = [c for c in drop_id_cols + drop_label_cols if c in df.columns]

# Ban đầu, lấy tất cả cột trừ mấy cột drop
feature_candidates = [c for c in df.columns if c not in cols_to_drop]
X_df = df[feature_candidates].copy()

# ---- XỬ LÝ block_timestamp TRƯỚC KHI LỌC NUMERIC ----
if "block_timestamp" in X_df.columns:
    # Trường hợp block_timestamp là chuỗi datetime -> convert sang datetime rồi sang int
    if not np.issubdtype(X_df["block_timestamp"].dtype, np.number):
        # cố gắng parse datetime; errors='coerce' nếu có ô lỗi sẽ thành NaT
        X_df["block_timestamp"] = pd.to_datetime(X_df["block_timestamp"], errors="coerce")
        # chuyển về "số giây từ epoch" (hoặc .view('int64') nếu bạn muốn ns)
        X_df["block_timestamp"] = X_df["block_timestamp"].astype("int64") // 10**9
    # nếu đã là số (unix time) thì thôi, giữ nguyên

# Giữ lại cột numeric để dùng cho ML
non_numeric = X_df.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    print("Bỏ cột không phải số:", non_numeric)
    X_df = X_df.drop(columns=non_numeric)

feature_cols = X_df.columns.tolist()
print("Feature dùng để train:", feature_cols)

X = X_df.values


Feature dùng để train: ['nonce', 'value', 'gas', 'gas_price', 'receipt_cumulative_gas_used', 'receipt_gas_used']


In [5]:
# ==========================
# 3b. Kiểm tra tương quan feature với label
# ==========================

# Ở đây ta dùng chính X_df (sau khi đã drop ID/label, xử lý timestamp, bỏ non-numeric)
# và cột nhãn LABEL_COL trong df.

# Gộp X_df (feature) với cột label thành một DataFrame chung
corr_df = pd.concat(
    [
        X_df.reset_index(drop=True),
        df[LABEL_COL].reset_index(drop=True)
    ],
    axis=1
)

# Tính ma trận tương quan Pearson
corr_matrix = corr_df.corr()

# Lấy vector tương quan của từng feature với label
corr_with_label = corr_matrix[LABEL_COL].drop(labels=[LABEL_COL])  # bỏ chính label

# Giá trị tuyệt đối để xem mức độ mạnh/yếu
corr_with_label_abs = corr_with_label.abs().sort_values(ascending=False)

print("\nTop 30 feature có |corr| lớn nhất với label:")
print(corr_with_label_abs.head(30))

# Nếu muốn xem cả dấu và |corr| dưới dạng bảng:
corr_table = pd.DataFrame({
    "corr": corr_with_label,
    "abs_corr": corr_with_label_abs
}).sort_values("abs_corr", ascending=False)

corr_table.head(30)



Top 30 feature có |corr| lớn nhất với label:
nonce                          0.120640
receipt_cumulative_gas_used    0.110460
receipt_gas_used               0.105110
gas_price                      0.048455
gas                            0.022670
value                          0.015716
Name: label, dtype: float64


,corr,abs_corr
nonce,-0.120640,0.120640
receipt_cumulative_gas_used,0.110460,0.110460
receipt_gas_used,0.105110,0.105110
gas_price,-0.048455,0.048455
gas,0.022670,0.022670
value,-0.015716,0.015716


In [6]:
# ==========================
# 4. Chia train / val / test (stratified)
# ==========================
TEST_SIZE = 0.15
VAL_SIZE  = 0.15

# B1: tách TEST
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

# B2: tách TRAIN & VAL từ phần còn lại
val_ratio_in_temp = VAL_SIZE / (1.0 - TEST_SIZE)  # ~0.1765

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=val_ratio_in_temp,
    random_state=RANDOM_STATE,
    stratify=y_temp
)

def show_stats(name, yy):
    counts = np.bincount(yy)
    n0 = counts[0] if len(counts) > 0 else 0
    n1 = counts[1] if len(counts) > 1 else 0
    ratio = n1 / (n0 + n1) if (n0 + n1) > 0 else 0
    print(f"{name:5s}: 0 = {n0:6d}, 1 = {n1:6d}, scam_ratio = {ratio:.6f}")

print("\n=== Phân bố nhãn sau khi chia ===")
show_stats("ALL",  y)
show_stats("Train", y_train)
show_stats("Val",   y_val)
show_stats("Test",  y_test)



=== Phân bố nhãn sau khi chia ===
ALL  : 0 =  57000, 1 =  14250, scam_ratio = 0.200000
Train: 0 =  39900, 1 =   9974, scam_ratio = 0.199984
Val  : 0 =   8550, 1 =   2138, scam_ratio = 0.200037
Test : 0 =   8550, 1 =   2138, scam_ratio = 0.200037


In [7]:
print("Feature dùng để train:", feature_cols)
print("Các cột nghi ngờ (scam/category/label):")
print([c for c in feature_cols
       if any(k in c.lower() for k in ["scam", "category", "label"])])

Feature dùng để train: ['nonce', 'value', 'gas', 'gas_price', 'receipt_cumulative_gas_used', 'receipt_gas_used']
Các cột nghi ngờ (scam/category/label):
[]


In [8]:
# ==========================
# 5. Chuẩn hóa feature
# ==========================
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

In [9]:
print("Train size:", X_train_scaled.shape, "Val size:", X_val_scaled.shape, "Test size:", X_test_scaled.shape)

# sanity check: không lẫn y
print("Số nhãn train:", np.bincount(y_train))
print("Số nhãn val  :", np.bincount(y_val))
print("Số nhãn test :", np.bincount(y_test))


Train size: (49874, 6) Val size: (10688, 6) Test size: (10688, 6)
Số nhãn train: [39900  9974]
Số nhãn val  : [8550 2138]
Số nhãn test : [8550 2138]


In [10]:
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.base import clone
from scipy.stats import randint, uniform

RANDOM_STATE = 42

def sample_param(dist, rng):
    """Lấy 1 giá trị từ distribution hoặc list."""
    if hasattr(dist, "rvs"):
        return dist.rvs(random_state=rng)
    # nếu là list/tuple
    dist = list(dist)
    return dist[rng.randint(0, len(dist))]

def random_search_single_model(name, base_estimator, param_dist,
                               X_train, y_train, X_val, y_val,
                               n_iter=30):
    print(f"\n===== Random search cho {name} (không k-fold, dùng VAL) =====")
    rng = np.random.RandomState(RANDOM_STATE)
    best_f1 = -1.0
    best_params = None

    for i in range(n_iter):
        # sample 1 bộ siêu tham số
        params = {k: sample_param(v, rng) for k, v in param_dist.items()}

        model = clone(base_estimator)
        model.set_params(**params)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_val_pred)

        print(f"Iter {i+1:02d}/{n_iter}: F1(val) = {f1:.4f}, params = {params}")

        if f1 > best_f1:
            best_f1 = f1
            best_params = params

    print(f"\n>>> {name} – best F1(val) = {best_f1:.4f}")
    print("Best params:", best_params)

    # Train lại trên TRAIN+VAL với best_params trước khi test
    X_train_full = np.vstack([X_train, X_val])
    y_train_full = np.concatenate([y_train, y_val])

    best_model = clone(base_estimator)
    best_model.set_params(**best_params)
    best_model.fit(X_train_full, y_train_full)

    return best_model


In [11]:
# Tính scale_pos_weight nếu muốn dùng cho XGB/LGBM
n_pos = np.sum(y_train == 1)
n_neg = np.sum(y_train == 0)
scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1.0
print("scale_pos_weight (train):", scale_pos_weight)

scale_pos_weight (train): 4.000401042711049


7. AutoML cho RF / XGB / LGBM

In [12]:

# 1) RandomForest
rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
rf_param_dist = {
    "n_estimators": randint(200, 600),
    "max_depth": randint(3, 30),
    "min_samples_split": randint(2, 50),
    "min_samples_leaf": randint(1, 20),
    "max_features": ["sqrt", "log2", None],
    "class_weight": [None, "balanced"]
}

best_rf = random_search_single_model(
    "RandomForest",
    rf_base,
    rf_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)


===== Random search cho RandomForest (không k-fold, dùng VAL) =====
Iter 01/30: F1(val) = 0.8398, params = {'n_estimators': 302, 'max_depth': 22, 'min_samples_split': 30, 'min_samples_leaf': 15, 'max_features': None, 'class_weight': 'balanced'}
Iter 02/30: F1(val) = 0.8332, params = {'n_estimators': 388, 'max_depth': 23, 'min_samples_split': 40, 'min_samples_leaf': 19, 'max_features': None, 'class_weight': None}
Iter 03/30: F1(val) = 0.8466, params = {'n_estimators': 287, 'max_depth': 23, 'min_samples_split': 37, 'min_samples_leaf': 8, 'max_features': None, 'class_weight': 'balanced'}
Iter 04/30: F1(val) = 0.6924, params = {'n_estimators': 508, 'max_depth': 4, 'min_samples_split': 25, 'min_samples_leaf': 12, 'max_features': 'log2', 'class_weight': 'balanced'}
Iter 05/30: F1(val) = 0.8318, params = {'n_estimators': 585, 'max_depth': 23, 'min_samples_split': 34, 'min_samples_leaf': 12, 'max_features': 'log2', 'class_weight': 'balanced'}
Iter 06/30: F1(val) = 0.8208, params = {'n_estimat

In [13]:
# 2) XGBoost
xgb_base = XGBClassifier(
    random_state=RANDOM_STATE,
    tree_method="hist",      # hoặc "gpu_hist" nếu bạn chạy GPU
    eval_metric="logloss",
    use_label_encoder=False
)
xgb_param_dist = {
    "n_estimators": randint(200, 800),
    "max_depth": randint(3, 12),
    "learning_rate": uniform(0.01, 0.29),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.0, 5.0),
    "scale_pos_weight": [scale_pos_weight]
}

best_xgb = random_search_single_model(
    "XGBoost",
    xgb_base,
    xgb_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)


===== Random search cho XGBoost (không k-fold, dùng VAL) =====


d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 01/30: F1(val) = 0.8495, params = {'n_estimators': 302, 'max_depth': 6, 'learning_rate': np.float64(0.28570714885887566), 'subsample': np.float64(0.892797576724562), 'colsample_bytree': np.float64(0.8394633936788146), 'min_child_weight': 7, 'gamma': np.float64(2.229163764267956), 'scale_pos_weight': np.float64(4.000401042711049)}


d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 02/30: F1(val) = 0.8546, params = {'n_estimators': 414, 'max_depth': 10, 'learning_rate': np.float64(0.10677549723031632), 'subsample': np.float64(0.6571467271687763), 'colsample_bytree': np.float64(0.8603553891795411), 'min_child_weight': 5, 'gamma': np.float64(4.8495492608099715), 'scale_pos_weight': np.float64(4.000401042711049)}


...

d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 29/30: F1(val) = 0.8376, params = {'n_estimators': 298, 'max_depth': 5, 'learning_rate': np.float64(0.2205271468723694), 'subsample': np.float64(0.9588441039810308), 'colsample_bytree': np.float64(0.954834569706047), 'min_child_weight': 8, 'gamma': np.float64(4.438850493804799), 'scale_pos_weight': np.float64(4.000401042711049)}


d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Iter 30/30: F1(val) = 0.8240, params = {'n_estimators': 786, 'max_depth': 3, 'learning_rate': np.float64(0.056872327087437995), 'subsample': np.float64(0.9594216754108317), 'colsample_bytree': np.float64(0.842571623863836), 'min_child_weight': 3, 'gamma': np.float64(1.8614138328087155), 'scale_pos_weight': np.float64(4.000401042711049)}

>>> XGBoost – best F1(val) = 0.8666
Best params: {'n_estimators': 458, 'max_depth': 9, 'learning_rate': np.float64(0.057750197411453104), 'subsample': np.float64(0.8136357677501768), 'colsample_bytree': np.float64(0.7939319885435933), 'min_child_weight': 1, 'gamma': np.float64(0.599326836668414), 'scale_pos_weight': np.float64(4.000401042711049)}


d:\elliptic\venv1\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [14]:
# 3) LightGBM
lgbm_base = LGBMClassifier(
    random_state=RANDOM_STATE,
    objective="binary",
    n_jobs=-1
)
lgbm_param_dist = {
    "n_estimators": randint(200, 800),
    "max_depth": randint(-1, 15),
    "num_leaves": randint(16, 256),
    "learning_rate": uniform(0.01, 0.29),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "min_child_samples": randint(5, 50),
    "scale_pos_weight": [scale_pos_weight]
}

best_lgbm = random_search_single_model(
    "LightGBM",
    lgbm_base,
    lgbm_param_dist,
    X_train_scaled, y_train,
    X_val_scaled,   y_val,
    n_iter=30
)


===== Random search cho LightGBM (không k-fold, dùng VAL) =====
[LightGBM] [Info] Number of positive: 9974, number of negative: 39900
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1499
[LightGBM] [Info] Number of data points in the train set: 49874, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199984 -> initscore=-1.386395
[LightGBM] [Info] Start training from score -1.386395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Iter 01/30: F1(val) = 0.7492, params = {'n_estimators': 302, 'max_depth': 2, 'num_leaves': 108, 'learning_rate': np.float64(0.0631960890611875), 'subsample': np.float64(0.9118764001091078), 'colsample_bytree': np.float64(0.8387400631785948), 'min_child_samples': 23, 'scale_pos_weight': np.float64(4.000401042711049)}
[LightGBM] [Info] Number of positive: 9974, number of negative: 39900
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1499
[LightGBM] [Info] Number of data points in the train set: 49874, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199984 -> initscore=-1.386395
[LightGBM] [Info] Start training from score -1.386395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Iter 02/30: F1(val) = 0.8830, params = {'n_estimators': 414, 'max_depth': 9, 'num_leaves': 218, 'learning_rate': np.float64(0.2611910822747312), 'subsample': np.float64(0.8404460046972835), 'colsample_bytree': np.float64(0.8832290311184181), 'min_child_samples': 26, 'scale_pos_weight': np.float64(4.000401042711049)}
[LightGBM] [Info] Number of positive: 9974, number of negative: 39900
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1499
[LightGBM] [Info] Number of data points in the train set: 49874, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199984 -> initscore=-1.386395
[LightGBM] [Info] Start training from score -1.386395


...

Iter 28/30: F1(val) = 0.8682, params = {'n_estimators': 419, 'max_depth': 4, 'num_leaves': 102, 'learning_rate': np.float64(0.09080747462861731), 'subsample': np.float64(0.9633063543866615), 'colsample_bytree': np.float64(0.695824756266789), 'min_child_samples': 6, 'scale_pos_weight': np.float64(4.000401042711049)}
[LightGBM] [Info] Number of positive: 9974, number of negative: 39900
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1499
[LightGBM] [Info] Number of data points in the train set: 49874, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199984 -> initscore=-1.386395
[LightGBM] [Info] Start training from score -1.386395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Iter 29/30: F1(val) = 0.8868, params = {'n_estimators': 252, 'max_depth': 10, 'num_leaves': 233, 'learning_rate': np.float64(0.2958386316920742), 'subsample': np.float64(0.6968221086046001), 'colsample_bytree': np.float64(0.8688542189623514), 'min_child_samples': 15, 'scale_pos_weight': np.float64(4.000401042711049)}
[LightGBM] [Info] Number of positive: 9974, number of negative: 39900
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1499
[LightGBM] [Info] Number of data points in the train set: 49874, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199984 -> initscore=-1.386395
[LightGBM] [Info] Start training from score -1.386395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [16]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)

models = {
    "RandomForest": best_rf,
    "XGBoost": best_xgb,
    "LightGBM": best_lgbm
}

for name, model in models.items():
    print(f"\n===== {name} trên TEST =====")
    y_pred_test = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred_test)
    f1_scam  = f1_score(y_test, y_pred_test)                    # mặc định: pos_label=1
    f1_micro = f1_score(y_test, y_pred_test, average="micro")
    f1_macro = f1_score(y_test, y_pred_test, average="macro")

    print("Accuracy:", acc)
    print("F1 (scam – binary, pos_label=1):", f1_scam)
    print("F1 micro:", f1_micro)
    print("F1 macro:", f1_macro)

    print("\nclassification_report:")
    print(classification_report(y_test, y_pred_test, digits=4))

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred_test))



===== RandomForest trên TEST =====
Accuracy: 0.9454528443113772
F1 (scam – binary, pos_label=1): 0.8650150497800416
F1 micro: 0.9454528443113772
F1 macro: 0.915417767019352

classification_report:
              precision    recall  f1-score   support

           0     0.9683    0.9634    0.9658      8550
           1     0.8565    0.8737    0.8650      2138

    accuracy                         0.9455     10688
   macro avg     0.9124    0.9186    0.9154     10688
weighted avg     0.9459    0.9455    0.9457     10688

Confusion matrix:
[[8237  313]
 [ 270 1868]]

===== XGBoost trên TEST =====
Accuracy: 0.9453592814371258
F1 (scam – binary, pos_label=1): 0.8679330619629128
F1 micro: 0.9453592814371258
F1 macro: 0.9167434567806778

classification_report:
              precision    recall  f1-score   support

           0     0.9739    0.9573    0.9656      8550
           1     0.8402    0.8976    0.8679      2138

    accuracy                         0.9454     10688
   macro avg     0

d:\elliptic\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
